# Library installation

In [3]:
install.packages('ggplot2')
install.packages('cowplot')
library('cowplot')
library('ggplot2')


  There is a binary version available but the source version is later:
        binary source needs_compilation
ggplot2  3.3.5  3.3.6             FALSE



installing the source package ‘ggplot2’





The downloaded binary packages are in
	/var/folders/q6/rqx7by952vl927kfl2fslt5h0000gn/T//Rtmp6F4R8w/downloaded_packages


In [4]:
setwd('/Users/Antoine/Desktop/Cours/Carlos III/Cours/Statistical learning/Assignement_1')
source("im2vect.R")

# Loading and preprocessing of the data

In [5]:
setwd('/Users/Antoine/Desktop/Cours/Carlos III/Cours/Statistical learning/Assignement_1/training')

In [6]:
# Creating a table of all the names of the images
D = NULL #matrix name BDD
for (k in 1:25){
  v = c(paste(k,"AT.jpg",sep=""),
        paste(k,"BT.jpg",sep=""),
        paste(k,"CT.jpg",sep=""),
        paste(k,"DT.jpg",sep=""),
        paste(k,"ET.jpg",sep=""),
        paste(k,"FT.jpg",sep=""))
  D = cbind(D,v)}

# Creating the training and validation data sets
n = dim(D) 
Data = NULL
index = NULL
Class = c()
for (j in 1:n[2]){ #across all people
  for (i in 1:(n[1])){ #across the images
    imacol = im2vect(D[i,j])
    Data = rbind(Data,imacol)
    index = rbind(index, D[i,j])#adding the names of the pictures as index and as a column
    Class = append(Class,j)
  }
}
rownames(Data) = index
Data = cbind(Data,Class)

# Principal component analysis

In [7]:
PCA = function(data){  # No class feature in data
  n = dim(data)
  mu = mean(data)
  G = data-mu
  SigmaS = 1/(n[1]-1)*G%*%t(G)
  eig = eigen(SigmaS) # calculing the eigen vectors and values of the small sigma
  eigvect = t(G)%*%eig$vectors # calculing the eigen vectors and values of the big sigma

  #Selecting the more important eigen vectors
  prop = cumsum(eig$values)/sum(eig$values)
  Number_of_eigen_vectors = 1:length(prop)

  ggplot(as.data.frame(prop), aes(x=Number_of_eigen_vectors , y=prop)) +
  geom_line(stat = "identity")+geom_point()+labs(title = "Proportion of variance reached")+ ylab("")
  
  N_PC =24 #number of eigenvectors choosen, 
  data.new = as.data.frame(data%*%eigvect[,1:N_PC])
  return(list('data'=data.new, 'eig'=eigvect[,1:N_PC])) 
}

# Fisher discriminant analysis

In [8]:
FDA = function(k, data,propvar){ # k the number of classes
  start_time <- Sys.time()
  
  m = colMeans(data[,1:ncol(data)-1]) # the mean matrix of the dataset's variables
  
  # Means of each classes
  names(data)[length(names(data))]<-"Class" # we rename the class column "Class"
  means = sapply(levels(data$Class), 
                 FUN = function(class){
                   colMeans(data[data$Class==class, 1:(ncol(data)-1)])
                 })
  
  # Scatter matrices
  S.W=data.frame(matrix(0, ncol = ncol(data)-1, nrow=ncol(data)-1)) 
  for (u in levels(data$Class)){
    Sw = cov(data[data$Class==u, 1:(ncol(data)-1)])*(table(data$Class)[1]-1)
    S.W = S.W + Sw
  }
  
  S.B=data.frame(matrix(0, ncol = ncol(data)-1, nrow=ncol(data)-1))
  for (u in 1:length(levels(data$Class))){
    Sb = (table(data$Class)[u])*(means[,u]-m)%*%t(means[,u]-m)
    S.B = S.B + Sb
  }
  
  # Eigen vectors and values
  eig = eigen(as.matrix(solve(S.W))%*%as.matrix(S.B))
  
  # Proportion of variance
  prop.var = eig$values/sum(eig$values)
  cummulative.var = cumsum(eig$values/sum(eig$values))
  i=1
  while (cummulative.var[i]<propvar){
    i=i+1
  }
  N_PC.FDA = i #determined after in the code 19
  
  eigenvector = eig$vectors[,1:N_PC.FDA]
  eigenvalues = eig$values[1:N_PC.FDA]
  
  end_time = Sys.time()
  data.new=as.data.frame(as.matrix(data[,1:ncol(data)-1])%*%eigenvector)
  return(list('mean'=m,'eig'=eigenvector,'prop'=prop.var,'data'=data.new))
}

# K nearest neighbors

In [9]:
Knn = function (im,data,k,eig.pca,eig.fda,meth){
  im = im2vect(im)
  im = proj(im,eig.pca,eig.fda)
  
  data.new = as.data.frame(rbind(im,data[,1:ncol(data)-1]))#add im to the dataset

  # Applying k-nn
  distance = dist(data.new, method=meth, diag=FALSE) #matrix of euclidian distance 
  distance = as.matrix(distance)
  distance = distance[2:nrow(distance),1]
  distance = cbind('distance'=distance,'Class'=data[,ncol(data)])#matrix with the label of the distance between im and each im of the database
  ord = distance[order(distance[,1]),]#tri de la matrice
  return(as.data.frame(ord[1:k,]))
}

# Projection

In [10]:
# Projection of a data into the new space, applying the PCA and Fischer
proj = function(im, eig.pca, eig.fda){
  im = as.matrix(t(im))%*%eig.pca #PCA
  im = im%*%eig.fda #FDA
  return(im)
}

# Distance metric

In [11]:
# Distance between an image and its dataset
ind = c()
ext = c()

for (i in 1:25){
  nd = sample(x=1:6,size=1) #choose 1 random image from the 6 of each person
  ind = append(ind,6*(i-1)+nd)
  ext = append(ext,D[nd,i]) #list of images extracted
}

# We drop one image of each person, this image is randomly choosen
data.train <- Data[-ind,]
data.test = Data[ind,]

In [12]:
# Method used to compute the distance
methode = c('euclidian','manhattan','canberra') 

sim = function(im, k, data.train, meth, propvar){ #return k closest neighbours and distance
  # Apply PCA to the dataset
  data.pca = PCA(data.train[,1:ncol(data.train)-1])
    
  # Apply FDA
  data.pca.class = cbind(data.pca$data,'Class'=as.factor(data.train[,ncol(data.train)]))#add the class vector
  data.fda = FDA(25,data.pca.class,propvar)
  data.fda.class = cbind(data.fda$data,'Class'=as.factor(data.train[,ncol(data.train)]))
    
  # Compute Knn
  tri = Knn(im,data.fda.class,k,data.pca$eig,data.fda$eig,meth)
  return(tri)
}

In [28]:
# Exemple
similarite = sim(ext[1], 8, data.train, methode[1], 0.99)
similarite

,distance,Class
,<dbl>,<dbl>
1FT.jpg,58.00817,1
1DT.jpg,89.11478,1
1ET.jpg,127.10885,1
1AT.jpg,128.65969,1
1BT.jpg,129.97231,1
9AT.jpg,1460.06763,9
9BT.jpg,1465.47199,9
9FT.jpg,1467.94411,9


# Prediction

In [14]:
# Guess the person thanks to the knn results  
guess = function(im, k, data.train, meth,propvar){
  similarite = sim(im,k,data.train,meth,propvar)
  tb = as.data.frame(table(similarite$Class))
  g = tb[which.max(tb$Freq),1]
  g = as.matrix(g)
  return(as.numeric(g))
}

# Full pipeline

In [25]:
# Final classifier
Classifier = function (name_im){
  similarite = sim(name_im, 6, data.train, 'euclidian', 0.99)
  if (similarite$distance[1]>450){
    g = 'the image doesn t belong to the dataset'
  }
  else{
    tb = as.data.frame(table(similarite$Class))
    g = tb[which.max(tb$Freq),1]
    g = as.matrix(g)
    g = as.numeric(g)
  }
  return(g)
}

# Evaluation

In [36]:
# Example
Classifier('1CT.jpg')

[1] 1

In [35]:
# Apply on all test dataset
result = NULL
for (k in rownames(data.test)){
    result = rbind(result, Classifier(k))
}
cbind(result, rownames(data.test))

1,1CT.jpg
2,2DT.jpg
3,3ET.jpg
4,4DT.jpg
5,5DT.jpg
6,6FT.jpg
7,7DT.jpg
8,8AT.jpg
9,9DT.jpg
10,10AT.jpg
11,11DT.jpg
